<a href="https://colab.research.google.com/github/james-t-little/bachelors_dissertation/blob/main/test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
from google.colab import drive
import matplotlib.pyplot as plt

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
IMG_SIZE = 512

def parse_image(image_path: str):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.uint8)
    return image

def normalize(input_image: tf.TensorArray): 
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image

def load_image_test(datapoint) -> tuple:
    input_image = tf.image.resize(datapoint, (IMG_SIZE, IMG_SIZE))
    input_image = normalize(input_image)
    return input_image

In [ ]:
test_dataset = tf.data.Dataset.list_files("/content/drive/MyDrive/dissertation_seg_data/test_eng_data/"+"*.png")

test_ds = test_dataset.map(parse_image)

test_ds = test_ds.map(load_image_test)
test_ds = test_ds.repeat()
test_ds = test_ds.batch(8)
test_ds = test_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [16]:
model = keras.models.load_model("/content/drive/MyDrive/"+"lr0001_e16_bs8.h5")

def display_sample(display_list):
    
    plt.figure(figsize=(18, 18))

    title = ['Input Image', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

def create_mask(pred_mask: tf.Tensor) -> tf.Tensor:
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = tf.expand_dims(pred_mask, axis=-1)
    return pred_mask

for image in test_ds.take(1):
    for i in image:
        pred_mask = model.predict(i)
        display_sample([i, create_mask(i)])

ValueError: ignored